In [1]:
import sys
sys.path.append("..")
import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols 
from rasterstats import zonal_stats
import rasterio
import contextily as cx
from rasterio.plot import show
#import osmnx as ox
from affine import Affine
import rioxarray as rx
from multiprocessing import Pool, cpu_count
from shapely.ops import unary_union
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from pathlib import Path
import fiona

#see all columns in tables
pd.set_option('display.max_columns', None)


%load_ext autoreload
%autoreload 2

In [2]:
from src.features.parcel_prep import parcel_ms4_model

Reading in municipal and regional boundaries...
Reading in land parcel database data...
Reading in public excluded land data...
Reading in additional data layers...
Wetlands...
Watersheds...
Wellhead protection areas...
Activity use limitation areas...
ParkServe data...


In [3]:
#list MAPC towns to include 
town_names = ['Reading', 'Winchester', 'Franklin', 'Natick', 'Norwood']
#town_names = ['Reading']

processed_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Output\\Parcels"
model_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Model"

In [50]:
#run suitability model for each town in list
for town_name in town_names:
    parcel_ms4_model(town_name, processed_path)


# MERGE SHAPEFILES 

filelist = []


# add all shapefiles 
for root, folder, files in os.walk(processed_path):
    for file in files:
        for muni in town_names:
            if muni in file:
                if file.endswith('.shp'):
                    fullname = os.path.join(root, file)
                    filelist.append(fullname)


merged_gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in filelist], 
                        ignore_index=True), crs=gpd.read_file(filelist[0]).crs)


#export
merged_gdf.to_file(model_path + '\\Merged_MS4_Parcels.shp')

Starting on Reading...
Prepping parcel data...
Prepping land cover data...
Calculating parcel score for each criteria...
Building and exporting combined data table...
Starting on Winchester...
Prepping parcel data...
Prepping land cover data...


KeyboardInterrupt: 

In [4]:
from src.features.ms4_funcs import get_parcels_row
from src.data.make_dataset import mapc_lpd
import imp
imp.reload(get_parcels_row)


town_parcels_row = get_parcels_row('Norwood', mapc_lpd)

220.0


In [5]:
town_parcels_row.sort_values(by='parloc_id').head()

,index,parloc_id,Address,Owner,UseCodes,UseDesc,TRANSIT,ACRES,SQFT,PublicInst,NonPubExc,Tot_Exclud,Tot_Sensit,ROW,OpenSpace,Hydrology,Wetlands,TitleV,Wellhead1,Flood_SHFA,Farmland,SurfWatBC,Wellhead2,IntWellhea,Habitat,geometry,OID_,mapc_id,muni_id,muni,poly_typ,loc_id_cnt,land_value,bldg_value,othr_value,total_value,ls_price,ls_date,bldg_area,res_area,luc_adj_1,luc_adj_2,num_units,units_est,units_src,num_rooms,yr_built,site_addr,addr_str,addr_num,owner_name,owner_addr,owner_city,owner_stat,fy,lot_areaft,far,pct_imperv,pct_bldg,pct_pave,landv_pac,bldgv_psf,totv_pac,bldlnd_rat,sqm_imperv,sqm_bldg,sqm_pave,realesttyp,temp,luc_adj_12,Description__short_,More_complete_description,ObjectID,Shape_Length,Shape_Area,non_exc,exc_acres,acreage,developable_land,pct_developable,type
8052,0,220.0_ROW_10000,NaN,ROW segment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((224874.193 880233.071, 224872.955 88...",NaN,NaN,NaN,Norwood,PRIV_ROW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ROW segment
8053,1,220.0_ROW_10001,NaN,ROW segment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((225872.112 884782.576, 225871.685 88...",NaN,NaN,NaN,Norwood,ROW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ROW segment
8054,2,220.0_ROW_10002,NaN,ROW segment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((226530.943 884505.836, 226530.648 88...",NaN,NaN,NaN,Norwood,ROW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ROW segment
8055,3,220.0_ROW_10003,NaN,ROW segment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((224731.777 884346.011, 224731.211 88...",NaN,NaN,NaN,Norwood,ROW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ROW segment
8056,4,220.0_ROW_10004,NaN,ROW segment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((225725.447 884416.378, 225724.574 88...",NaN,NaN,NaN,Norwood,ROW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ROW segment


In [55]:
row_gdb = 'K:\DataServices\Projects\Current_Projects\Environment\MS4\Project\RightOfWay_Segmentation.gdb'
print('reading in eot roads')
eot_road_mapc = gpd.read_file(row_gdb, layer='EOTROADS_MAPC', mask=muni_shp)

join_roads = town_parcels_row.sjoin(eot_road_mapc, how="left")
join_roads.head()

[autoreload of src.features.ms4_funcs failed: Traceback (most recent call last):
  File "c:\Users\rbowers\Anaconda3\envs\ms4_env\lib\site-packages\IPython\extensions\autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\rbowers\Anaconda3\envs\ms4_env\lib\site-packages\IPython\extensions\autoreload.py", line 459, in superreload
    module = reload(module)
  File "c:\Users\rbowers\Anaconda3\envs\ms4_env\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\rbowers\scripting_projects\ms4-gi-equity-tool\notebooks\..\src\features\ms4_funcs.py", line 36, in <module>
    eot_road_mapc = gpd.read_file(row_gdb, layer='EOTROADS_MAPC')
  File "c:\Users\rbowers\Anaconda3\envs\ms4_env\lib\site-packages\

In [8]:
from src.features.parcel_prep import tess_ms4_model

town_names = ['Natick']
processed_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Output\\Tess"

#run suitability model for each town in list
for town_name in town_names:
    tess_ms4_model(town_name, processed_path)

Starting on Natick...
Prepping parcel data...
Prepping land cover data...
Calculating parcel score for each criteria...
Building and exporting combined data table...


In [15]:
town_name = 'Natick'
processed_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Output\\Tess"
path = os.path.join(processed_path, town_name) #make a subdirectory in intermediate folder w town name
os.makedirs(path, exist_ok=True)
path_2 = path + '\\' + town_name + '_MS4_GI_tesselation.shp'

natick_tess = gpd.read_file(path_2)

In [13]:
from src.features.ms4_funcs import get_parcels_row
from src.data.make_dataset import mapc_lpd

natick_parcels_row = get_parcels_row('Natick', mapc_lpd)

In [ ]:
natick_parcels_row_tess = natick_tess.sjoin()

In [137]:
from src.features.ms4_funcs import get_parcels_row
from src.features.build_features import *


,GRID_ID,GROUP_,pblc,ROW_ID,row,sqm_imp_cvr,pct_imp_cvr,sqm_imp_rf,pct_imp_rf,sqm_imprv,pct_imprv,sqm_prv,pct_prv,K_sum,K_per_acre,ParkRank,parks,IT_VALDESC,wtlnds,SUPPLIER_x,z1_wpa,SUPPLIER_y,z2_wpa,SUPPLIER,int_wpa,aul,NAME,NAME_first,rnk_tree,tree,rnk_heat,heat,EJ_CRIT_DESC,ej,geometry
0,MO-420,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,0,0,0,0,0,0,0,NaN,CHARLES,0.0,0,0.0,0,0,0,"POLYGON ((210190.273 867568.145, 210194.648 86..."
1,HM-519,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,0,0,0,0,0,0,0,NaN,CHARLES,0.0,0,0.0,0,0,0,"POLYGON ((207739.680 865451.242, 207752.088 86..."
2,HN-519,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,0,0,0,0,0,0,0,NaN,CHARLES,0.0,0,0.0,0,0,0,"POLYGON ((207775.860 865449.434, 207754.628 86..."
3,HO-518,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132.600003,1341.580245,1.0,1,0,0,0,0,0,0,0,0,0,NaN,CHARLES,0.0,0,0.0,0,0,0,"POLYGON ((207789.312 865451.242, 207776.904 86..."
4,HM-518,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1,0,0,0,0,0,0,0,0,0,NaN,CHARLES,0.0,0,0.0,0,0,0,"POLYGON ((207752.088 865451.242, 207739.680 86..."


### Identify muni/munis of interest, create a shapefile for the muni

Then create land use and land cover layers of interest based on the municipal boundary mask (speeds up processes)

### Pull base geographies for analysis
- Get municipal ROW and fishnet (tesselation) layers from geodatabases 
- Clean up those gdfs to include only the relevant info

`town_parcels_row` and `town_tess` are the two base geographies for this analysis.

## Inidicator Layer Prep 
To improve run time, for some indicators it is better to read in here with a municipal mask 

2023.06.1 - can delete most of this except the fishnet stuff

In [86]:
from src.data.make_dataset import *
from src.features.build_features import *
from src.features.ms4_funcs import *
#select a town name, create a gdf of that town boundary

town_name = 'Franklin'


muni_shp = munis.loc[munis['municipal'] == town_name]
#read in relevant data layers
from shapely.validation import make_valid

row_gdb = 'K:\DataServices\Projects\Current_Projects\Environment\MS4\Project\ROW_model_output.gdb'

for layer_name in fiona.listlayers(row_gdb):
    if town_name in layer_name:
        town_row_layer = layer_name


town_row = gpd.read_file(row_gdb, layer=town_row_layer)
town_row = town_row.explode().reset_index(drop=True)

##FISHNET##

fishnets_gdb = 'K:\\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Fishnets.gdb'

for layer_name in fiona.listlayers(fishnets_gdb):
    if town_name in layer_name:
        town_tess = layer_name

muni_tess = gpd.read_file(fishnets_gdb, layer=town_tess)
#muni_tess = gpd.read_file(muni_tess_gdb, layer='muni_tessellation')
#clip tesselation to the franklin boundary
muni_tess = muni_tess.clip(muni_shp)
muni_tess['sqm'] = muni_tess['geometry'].area

#create an imperviousness layer
#first read in lclu with a muni shapefile mask

ms4_model_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Model.gdb'
lclu = gpd.read_file(ms4_model_gdb, layer="lclu_simplify_all_mapc", mask=muni_shp)

#fix any geometry issues
print('Validating land use geometry...')
lclu.geometry = lclu.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)
lclu = lclu.clip(muni_shp)

#get phosphorous load per land cover geometry area
lclu['acres'] = lclu['geometry'].area / 4047
lclu['K_load'] = lclu['pler'] * lclu['acres']

#select only for imperviousness land cover codes
lclu_impervious = lclu.loc[lclu['covercode'] == 2]

#fix any geometry issues
print('Validating imperviousness geometry...')
lclu_impervious.geometry = lclu_impervious.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)

#tree canopy using land use
tree_canopy_covernames = ['Deciduous Forest', 'Evergreen Forest', 'Palustrine Forested Wetland']
lclu_treecanopy = lclu.loc[lclu['covername'].isin(tree_canopy_covernames)]

print('Validating tree canopy geometry...')
lclu_treecanopy.geometry = lclu_treecanopy.apply(lambda row: make_valid(row.geometry) if not row.geometry.is_valid else row.geometry, axis=1)

#fix geom inconsistnecies
lclu_treecanopy = lclu_treecanopy.explode()
lclu_treecanopy = lclu_treecanopy.loc[lclu_treecanopy['geometry'].geom_type == 'Polygon']

#make a layer that gives us impervious surfaces but distinguishes between impervious land cover vs rooftops

#read in structures for the muni
building_structures_gdb = 'K:\\DataServices\\Datasets\\MassGIS\\Facilities_Structures\\Building_Structures\\Output\\structures.gdb'
building_structures = gpd.read_file(building_structures_gdb, layer='STRUCTURES_POLY', mask=muni_shp)

#add a type field
building_structures['type'] = 'rooftop'

#erase structures from the imperviousness lclu layer
imperv_cover = lclu_impervious.overlay(building_structures, how='difference')

#now we just have land cover (not rooftops) that are imperviousness
#add a type field
imperv_cover['type'] = 'land cover'

#join back together with the 'type' field being the distinguisher btwn land cover and rooftops
imperviousness_by_type = pd.concat([building_structures[['type', 'geometry']],
                                    imperv_cover[['type', 'geometry']]
                                    ])

#dissolve by type
imperviousness_by_type = imperviousness_by_type.dissolve(by='type')

#add a "layer" field for the model
imperviousness_by_type['layer'] = 'impervious'

#get tree "need" layer (block group)
muni_tree_need = tree_score(muni_shp, lclu_treecanopy)
muni_tree_need.head()

#get heat "vulnerability" layer (block)
muni_heat_vuln = heat_score(muni_shp, heat_fp)

#get soil hydrology score
muni_soils_hsg = soil_hsg_score(muni_shp)


Validating land use geometry...
Validating imperviousness geometry...
Validating tree canopy geometry...


In [155]:
town_name = 'Franklin'

drainage_network_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\Drainage_network.gdb'

for layer_name in fiona.listlayers(drainage_network_gdb):
    if town_name in layer_name:
        if 'pts' in layer_name:
            print(layer_name)



Franklin_pts


## Tesselation Analysis

In [114]:
#does it overlap with public land
print("Calculating public land overlap...")
public_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=public_land, 
                                            field='GROUP_',
                                            layer_name='pblc',
                                            overlap=0.05)

print("Calculating right of way overlap...")
town_row['ROW_ID'] = town_row['parloc_id']
row_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=town_row, 
                                            field='ROW_ID',
                                            layer_name='row',
                                            overlap=0.05)

print("Calculating imperviousness overlap...")
imperv_tess = calculate_imperviousness(muni_tess, 
                                  'GRID_ID', 
                                  imperv_cover, 
                                  building_structures)

print('Calculating pler...')
#what is the imperviousness pler on site?
pler_tess = get_K_load(parcel_data=muni_tess,
                         id_field = 'GRID_ID',
                         muni_name = town_name,
                         muni_shpf = muni_shp,
                         lclu_layer=lclu_impervious,
                         pler_field='pler')


print("Calculating park serve overlap...")
#does it overlap with Trust for Public Land's "Park Serve" priority areas from 2022
parkserve_tess = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='GRID_ID',
                                        parcel_data=muni_tess, 
                                        join_data=parkserve_data, 
                                        field='ParkRank',
                                        layer_name='parks',
                                        overlap=0.05)

print("Calculating wetland overlap...")
#does it overlap with wetlands?
wetlands_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=wetlands, 
                                            field='IT_VALDESC',
                                            layer_name='wtlnds',
                                            overlap=0.05)
print("Calculating wpa overlap...")
#does it overlap with wellhead protection areas?
wpa_z1_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=zone1_wpa, 
                                            field='SUPPLIER',
                                            layer_name='z1_wpa',
                                            overlap=0.05)

wpa_z2_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=zone2_wpa, 
                                            field='SUPPLIER',
                                            layer_name='z2_wpa',
                                            overlap=0.05)


wpa_interim_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=interim_wpa, 
                                            field='SUPPLIER',
                                            layer_name='int_wpa',
                                            overlap=0.05)

print("Calculating aul overlap...")
#does it have an activity use limitation area within it?
aul_ovlp_tess = calculate_suitability_criteria(how='if_overlap', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            points=True,
                                            join_data=aul, 
                                            field='NAME',
                                            layer_name='aul',
                                            overlap=0.05)

print("Calculating watershed overlap...")
#what watershed does it overlap with?
watershed_tess = calculate_suitability_criteria(how='overlap_sjoin', 
                                            id_field='GRID_ID',
                                            parcel_data=muni_tess, 
                                            join_data=major_basins, 
                                            stats='first',
                                            field='NAME',
                                            layer_name='wtshd')

print("Calculating tree need overlap...")
#is it in a block group with tree need?
tree_need_tess = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='GRID_ID',
                                        parcel_data=muni_tess, 
                                        join_data=muni_tree_need, 
                                        field='rnk_tree',
                                        layer_name='tree',
                                        overlap=0.05)

print("Calculating heat vulnerability overlap...")
#is it in a census block with high heat vulnerability?
heat_vln_tess = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='GRID_ID',
                                        parcel_data=muni_tess, 
                                        join_data=muni_heat_vuln, 
                                        field='rnk_heat',
                                        layer_name='heat',
                                        overlap=0.05)

print("Calculating ej block group overlap...")
#is it in an ej community?
ej_tess = calculate_suitability_criteria(how='if_overlap', 
                                        id_field='GRID_ID',
                                        parcel_data=muni_tess, 
                                        join_data=ej_2020, 
                                        field='EJ_CRIT_DESC',
                                        layer_name='ej',
                                        overlap=0.05)


Calculating public land overlap...
Calculating right of way overlap...
Calculating imperviousness overlap...
Calculating pler...
Calculating park serve overlap...
Calculating wetland overlap...
Calculating wpa overlap...
Calculating aul overlap...
Calculating watershed overlap...
Calculating tree need overlap...
Calculating heat vulnerability overlap...
Calculating ej block group overlap...


In [115]:

#then join parcel data to the tesselations. Each tessel may have more than one associated 
#this might be more relevant later
'''muni_tess_parcels = muni_tess.overlay(town_parcels_row[['parloc_id', 
                                                        'Address', 
                                                        'Owner', 
                                                        'UseDesc', 
                                                        'muni', 
                                                        'type', 
                                                        'acreage', 
                                                        'geometry']], 
                                                        how='intersection')'''
'''
muni_tess_parcels_agg = muni_tess_parcels[['GRID_ID', 'sqm', 'parloc_id', 'type']].groupby(
    ['GRID_ID'], as_index=False).agg(
        {'parloc_id':list, 'type':list})


muni_tess_parcels_agg = muni_tess.merge(muni_tess_parcels_agg, on='GRID_ID')'''

dfs = [ public_tess,
       row_tess,
       imperv_tess,
       pler_tess,
       parkserve_tess,
       wetlands_ovlp_tess,
       wpa_z1_ovlp_tess,
       wpa_z2_ovlp_tess,
       wpa_interim_ovlp_tess,
       aul_ovlp_tess,
       watershed_tess,
       tree_need_tess,
       heat_vln_tess,
       ej_tess
    ]

 #merge all of the listed dataframes together based on their parcel id and geometry
df_merged = reduce(lambda  left,right: pd.merge(left,
                                                right,
                                                on=['GRID_ID', 'geometry'], 
                                                how='outer'), 
                                                dfs)

df_merged.insert((len(df_merged.columns) - 1), 'geometry', df_merged.pop('geometry'))

#run suitability model for each town in list
processed_path = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Data\\Spatial\\Output"

#add back merged shapefile (add this back later)

filelist = []

#export
df_merged.to_file(processed_path + '\\Test_Combined_Shapefile_MS4_gi_equity_tess.shp')
